In [19]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [20]:
train_data = pd.read_csv(r"C:\Users\aniqa\Desktop\Courses\Semester_2\Machine_Learning\Project 1\Code\train_data.csv", nrows=1440000)
train_labels_data = pd.read_csv(r"C:\Users\aniqa\Desktop\Courses\Semester_2\Machine_Learning\Project 1\Code\train_labels.csv")

#data = train_data.sample(frac=1)
#train_data_sample = data.groupby('customer_ID',as_index=False).first()

#print(data2)
print(train_data.shape)
print(train_labels_data.shape)
train_data.shape
train_labels_data.shape
#print(train_data[1])

(1440000, 190)
(458913, 2)


(458913, 2)

In [21]:
print(train_data.head)

<bound method NDFrame.head of                                                customer_ID         S_2  \
0        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-03-09   
1        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-04-07   
2        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-05-28   
3        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-06-13   
4        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-07-16   
...                                                    ...         ...   
1439995  425d5532a25f20815166956bc7fb3a49928f4c49517e1c...  2017-06-29   
1439996  425d5532a25f20815166956bc7fb3a49928f4c49517e1c...  2017-07-29   
1439997  425d5532a25f20815166956bc7fb3a49928f4c49517e1c...  2017-08-30   
1439998  425d5532a25f20815166956bc7fb3a49928f4c49517e1c...  2017-09-29   
1439999  425d5532a25f20815166956bc7fb3a49928f4c49517e1c...  2017-10-30   

              P_2      D_39       B_1       B_2       R_1       S_3      D_41  \


In [22]:
train_data['S_2'] = pd.to_datetime(train_data['S_2'])
#data = data.groupby('customer_ID')
train_data.tail(5)


,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
1439995,425d5532a25f20815166956bc7fb3a49928f4c49517e1c...,2017-06-29,0.874078,0.004086,0.008672,0.815791,0.005246,NaN,0.008112,0.006125,...,NaN,NaN,NaN,0.002196,0.006976,0.004546,NaN,0.009514,0.002603,0.009218
1439996,425d5532a25f20815166956bc7fb3a49928f4c49517e1c...,2017-07-29,0.872043,0.007395,0.001348,0.819464,0.004784,NaN,0.009488,0.008950,...,NaN,NaN,NaN,0.002335,0.005112,0.003713,NaN,0.000811,0.008883,0.001153
1439997,425d5532a25f20815166956bc7fb3a49928f4c49517e1c...,2017-08-30,0.867252,0.008150,0.008059,0.818707,0.007366,NaN,0.005170,0.000103,...,NaN,NaN,NaN,0.005987,0.008728,0.004927,NaN,0.007416,0.006252,0.002668
1439998,425d5532a25f20815166956bc7fb3a49928f4c49517e1c...,2017-09-29,0.934603,0.002312,0.006102,0.816108,0.003532,NaN,0.000250,0.000844,...,NaN,NaN,NaN,0.006903,0.004097,0.009254,NaN,0.005879,0.002002,0.005971
1439999,425d5532a25f20815166956bc7fb3a49928f4c49517e1c...,2017-10-30,0.938461,0.005231,0.000666,0.810569,0.007312,NaN,0.004932,0.002325,...,NaN,NaN,NaN,0.008898,0.006947,0.008566,NaN,0.001507,0.001307,0.009375


In [23]:
last_customer_id = train_data['customer_ID'].iloc[-1]
print(last_customer_id)

#data2 = train_data_sample[train_data_sample['customer_ID'] != '425d5532a25f20815166956bc7fb3a49928f4c49517e1c59f9843f13a8aad690']
#print(data2.tail)

425d5532a25f20815166956bc7fb3a49928f4c49517e1c59f9843f13a8aad690


In [24]:
data2 = train_data[train_data['customer_ID'] != last_customer_id]
print(data2.tail)

<bound method NDFrame.tail of                                                customer_ID        S_2  \
0        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f... 2017-03-09   
1        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f... 2017-04-07   
2        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f... 2017-05-28   
3        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f... 2017-06-13   
4        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f... 2017-07-16   
...                                                    ...        ...   
1439987  425d3742a1fd67755c431a132257d18e81234fc36985cd... 2017-11-02   
1439988  425d3742a1fd67755c431a132257d18e81234fc36985cd... 2017-12-22   
1439989  425d3742a1fd67755c431a132257d18e81234fc36985cd... 2018-01-18   
1439990  425d3742a1fd67755c431a132257d18e81234fc36985cd... 2018-02-18   
1439991  425d3742a1fd67755c431a132257d18e81234fc36985cd... 2018-03-12   

              P_2      D_39       B_1       B_2       R_1       S_3      D_41  \
0        0.9

In [25]:
data2.shape

(1439992, 190)

In [26]:
merged_data = pd.merge(data2, train_labels_data, on='customer_ID').dropna(subset=['target'])
print(merged_data.shape)

(1439992, 191)


In [27]:
merged_data.to_csv(r"C:\Users\aniqa\Desktop\Courses\Semester_2\Machine_Learning\Project 1\Code\merged_data_nn.csv", index = False)

In [28]:
data_types = merged_data.dtypes
data_types = pd.DataFrame({"label": merged_data.columns,"type": data_types})
print(data_types)

                   label            type
customer_ID  customer_ID          object
S_2                  S_2  datetime64[ns]
P_2                  P_2         float64
D_39                D_39         float64
B_1                  B_1         float64
...                  ...             ...
D_142              D_142         float64
D_143              D_143         float64
D_144              D_144         float64
D_145              D_145         float64
target            target           int64

[191 rows x 2 columns]


In [29]:
categorical_variables = data_types.loc[data_types['type']=='object']
print(categorical_variables)

                   label    type
customer_ID  customer_ID  object
D_63                D_63  object
D_64                D_64  object


In [30]:
categorical_variables = categorical_variables.iloc[1:]
print(categorical_variables)

     label    type
D_63  D_63  object
D_64  D_64  object


In [31]:
cat_list = categorical_variables.iloc[:, 0].tolist()
print(cat_list)

['D_63', 'D_64']


In [32]:
one_hot = OneHotEncoder(handle_unknown = 'ignore')
one_hot.fit(merged_data[cat_list])
merged_data_encoded = pd.DataFrame(one_hot.transform(merged_data[cat_list]).toarray(),columns = one_hot.get_feature_names_out(cat_list))
merged_data = pd.concat([merged_data.reset_index(drop = True), merged_data_encoded.reset_index(drop=True)],axis = 1)
print(merged_data)

                                               customer_ID        S_2  \
0        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f... 2017-03-09   
1        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f... 2017-04-07   
2        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f... 2017-05-28   
3        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f... 2017-06-13   
4        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f... 2017-07-16   
...                                                    ...        ...   
1439987  425d3742a1fd67755c431a132257d18e81234fc36985cd... 2017-11-02   
1439988  425d3742a1fd67755c431a132257d18e81234fc36985cd... 2017-12-22   
1439989  425d3742a1fd67755c431a132257d18e81234fc36985cd... 2018-01-18   
1439990  425d3742a1fd67755c431a132257d18e81234fc36985cd... 2018-02-18   
1439991  425d3742a1fd67755c431a132257d18e81234fc36985cd... 2018-03-12   

              P_2      D_39       B_1       B_2       R_1       S_3      D_41  \
0        0.938469  0.001733  0.008724  1.0

In [33]:
#Missing value imputation
merged_data.fillna(0,inplace=True)


In [34]:
print(merged_data['S_2'].min())
print(merged_data['S_2'].max())

2017-03-01 00:00:00
2018-03-31 00:00:00


In [35]:
train_data = merged_data[(merged_data['S_2']>='2017-05-01') & (merged_data["S_2"] <= '2018-01-31')]
test1_data = merged_data[(merged_data['S_2']>='2017-03-01') & (merged_data['S_2'] <= '2017-04-30')]
test2_data = merged_data[(merged_data['S_2']>='2018-02-01') & (merged_data['S_2'] <= '2018-03-31')]

In [36]:
print(train_data['S_2'].min())
print(train_data['S_2'].max())
print(test1_data['S_2'].min())
print(test1_data['S_2'].max())
print(test2_data['S_2'].min())
print(test2_data['S_2'].max())

2017-05-01 00:00:00
2018-01-31 00:00:00
2017-03-01 00:00:00
2017-04-30 00:00:00
2018-02-01 00:00:00
2018-03-31 00:00:00


Get the feature importance data saved from XGBoost model. We will only run normalization and outlier treatment on these features.

In [37]:
merged_importance_df=pd.read_csv(r"C:\Users\aniqa\Desktop\Courses\Semester_2\Machine_Learning\Project 1\Code\merged_importance.csv")

In [38]:
# Create a boolean mask for features with importance greater than 0.5% in either column
mask = (merged_importance_df['importance_1'] > 0.005) | (merged_importance_df['importance_2'] > 0.005)

# Get the feature names where the mask is True
important_features = merged_importance_df.loc[mask, 'feature_name'].tolist()
print(len(important_features))

41


In [39]:
target_col = 'target'
drop_cols = ['customer_ID','S_2', 'D_63','D_64',target_col]

X_train = train_data.drop(columns = drop_cols)
X_train = X_train[important_features]
Y_train = train_data[target_col]

X_test1 = test1_data.drop(columns = drop_cols)
X_test1 = X_test1[important_features]
Y_test1 = test1_data[target_col]

X_test2 = test2_data.drop(columns = drop_cols)
X_test2 = X_test2[important_features]
Y_test2 = test2_data[target_col]

In [40]:
print(X_train.shape)
print(Y_train.sum())

(995264, 41)
249910


In [41]:
#Normalization
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)

StandardScaler()

In [42]:
X_train_normalized = sc.transform(X_train)
X_test1_normalized = sc.transform(X_test1)
X_test2_normalized = sc.transform(X_test2)

In [43]:
X_train_normalized = pd.DataFrame(X_train_normalized, columns=X_train.columns)
X_test1_normalized = pd.DataFrame(X_test1_normalized, columns=X_test1.columns)
X_test2_normalized = pd.DataFrame(X_test2_normalized, columns=X_test2.columns)

In [44]:
#Outlier Treatment for train sample
X_train_normalized.describe(percentiles=[0.01, 0.99]).transpose()

for col in X_train_normalized.columns:
    # calculate the 1% and 99% quantiles for the column
    min_val = X_train_normalized[col].quantile(0.01)
    max_val = X_train_normalized[col].quantile(0.99)
    
    #replace any values outside of the range with the corresponding minimum or maximum value
    X_train_normalized[col] = X_train_normalized[col].apply(lambda x: min_val if x < min_val else max_val if x > max_val else x)

In [45]:
#Outlier Treatment for test sample 1
X_test1_normalized.describe(percentiles=[0.01, 0.99]).transpose()

for col in X_train_normalized.columns:
    # calculate the 1% and 99% quantiles for the column
    min_val = X_test1_normalized[col].quantile(0.01)
    max_val = X_test1_normalized[col].quantile(0.99)
    
    #replace any values outside of the range with the corresponding minimum or maximum value
    X_test1_normalized[col] = X_test1_normalized[col].apply(lambda x: min_val if x < min_val else max_val if x > max_val else x)

In [46]:
#Outlier Treatment for test sample 2
X_test2_normalized.describe(percentiles=[0.01, 0.99]).transpose()

for col in X_train_normalized.columns:
    # calculate the 1% and 99% quantiles for the column
    min_val = X_test2_normalized[col].quantile(0.01)
    max_val = X_test2_normalized[col].quantile(0.99)
    
    #replace any values outside of the range with the corresponding minimum or maximum value
    X_test2_normalized[col] = X_test2_normalized[col].apply(lambda x: min_val if x < min_val else max_val if x > max_val else x)

In [48]:
import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import Dense

In [51]:
# set hyperparameters
num_hidden_layers = [2, 4]
num_nodes = [4, 6]
activation_funcs = ['relu', 'tanh']
dropout_rates = [0.5, 1.0]
batch_sizes = [100, 10000]
num_epochs = 20

# create empty dataframe to store results
results_df = pd.DataFrame(columns=['# HL', '#Node', 'Activation Function', 'Dropout', 'Batch Size', 'AUC Train', 'AUC Test 1', 'AUC Test 2'])

# iterate through all combinations of hyperparameters
for hl in num_hidden_layers:
    for nn in num_nodes:
        for af in activation_funcs:
            for dr in dropout_rates:
                for bs in batch_sizes:
                    
                    # build neural network model
                    model = Sequential()
                    model.add(Dense(nn, activation=af, input_shape=(X_train_normalized.shape[1],)))
                    for i in range(hl - 1):
                        model.add(Dense(nn, activation=af))
                        if dr < 1.0:
                            model.add(Dropout(dr))
                    model.add(Dense(1, activation='sigmoid'))
                    model.compile(loss='binary_crossentropy', optimizer='adam')
                    
                    # train model
                    model.fit(X_train_normalized, Y_train, batch_size=bs, epochs=num_epochs, verbose=0)
                    
                    # evaluate performance on train and test sets
                    y_pred_train = model.predict(X_train_normalized).ravel()
                    y_pred_test1 = model.predict(X_test1_normalized).ravel()
                    y_pred_test2 = model.predict(X_test2_normalized).ravel()
                    auc_train = roc_auc_score(Y_train, y_pred_train)
                    auc_test1 = roc_auc_score(Y_test1, y_pred_test1)
                    auc_test2 = roc_auc_score(Y_test2, y_pred_test2)
                    
                    # add results to dataframe
                    results_df = results_df.append({'# HL': hl, '#Node': nn, 'Activation Function': af, 'Dropout': dr, 'Batch Size': bs, 'AUC Train': auc_train, 'AUC Test 1': auc_test1, 'AUC Test 2': auc_test2}, ignore_index=True)
                    print(results_df.iloc[-1])
                    # save results to csv after each iteration
                    #results_df.to_csv('results.csv', index=False)

7413/7413 [==============================] - 5s 608us/step


C:\Users\aniqa\AppData\Local\Temp\ipykernel_13788\3386485280.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'# HL': hl, '#Node': nn, 'Activation Function': af, 'Dropout': dr, 'Batch Size': bs, 'AUC Train': auc_train, 'AUC Test 1': auc_test1, 'AUC Test 2': auc_test2}, ignore_index=True)


# HL                          2
#Node                         4
Activation Function        relu
Dropout                     0.5
Batch Size                  100
AUC Train              0.931983
AUC Test 1             0.916454
AUC Test 2              0.95054
Name: 0, dtype: object
7413/7413 [==============================] - 4s 605us/step


C:\Users\aniqa\AppData\Local\Temp\ipykernel_13788\3386485280.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'# HL': hl, '#Node': nn, 'Activation Function': af, 'Dropout': dr, 'Batch Size': bs, 'AUC Train': auc_train, 'AUC Test 1': auc_test1, 'AUC Test 2': auc_test2}, ignore_index=True)


# HL                          2
#Node                         4
Activation Function        relu
Dropout                     0.5
Batch Size                10000
AUC Train              0.930433
AUC Test 1             0.914649
AUC Test 2             0.949792
Name: 1, dtype: object
7413/7413 [==============================] - 4s 596us/step


...

C:\Users\aniqa\AppData\Local\Temp\ipykernel_13788\3386485280.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'# HL': hl, '#Node': nn, 'Activation Function': af, 'Dropout': dr, 'Batch Size': bs, 'AUC Train': auc_train, 'AUC Test 1': auc_test1, 'AUC Test 2': auc_test2}, ignore_index=True)


# HL                          4
#Node                         6
Activation Function        tanh
Dropout                     0.5
Batch Size                10000
AUC Train              0.930136
AUC Test 1             0.913926
AUC Test 2              0.94882
Name: 29, dtype: object
7413/7413 [==============================] - 5s 624us/step


C:\Users\aniqa\AppData\Local\Temp\ipykernel_13788\3386485280.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'# HL': hl, '#Node': nn, 'Activation Function': af, 'Dropout': dr, 'Batch Size': bs, 'AUC Train': auc_train, 'AUC Test 1': auc_test1, 'AUC Test 2': auc_test2}, ignore_index=True)


# HL                          4
#Node                         6
Activation Function        tanh
Dropout                     1.0
Batch Size                  100
AUC Train              0.933439
AUC Test 1             0.917751
AUC Test 2             0.950986
Name: 30, dtype: object
7413/7413 [==============================] - 5s 610us/step
# HL                          4
#Node                         6
Activation Function        tanh
Dropout                     1.0
Batch Size                10000
AUC Train              0.931947
AUC Test 1             0.916112
AUC Test 2              0.95018
Name: 31, dtype: object


C:\Users\aniqa\AppData\Local\Temp\ipykernel_13788\3386485280.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'# HL': hl, '#Node': nn, 'Activation Function': af, 'Dropout': dr, 'Batch Size': bs, 'AUC Train': auc_train, 'AUC Test 1': auc_test1, 'AUC Test 2': auc_test2}, ignore_index=True)


In [52]:
results_df.to_csv(r"C:\Users\aniqa\Desktop\Courses\Semester_2\Machine_Learning\Project 1\Code\NN_grid_results.csv")

In [53]:
#set best hyperparameters
hl = 4
nn = 4
af = 'tanh'
dr = 1.0
bs = 100

#fit the 
model = Sequential()
model.add(Dense(nn, activation=af, input_shape=(X_train_normalized.shape[1],)))
for i in range(hl - 1):
    model.add(Dense(nn, activation=af))
    if dr < 1.0:
        model.add(Dropout(dr))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')              
model.fit(X_train_normalized, Y_train, batch_size=bs, epochs=20, verbose=0)

In [54]:
y_pred_train = model.predict(X_train_normalized).ravel()
y_pred_test1 = model.predict(X_test1_normalized).ravel()
y_pred_test2 = model.predict(X_test2_normalized).ravel()
auc_train = roc_auc_score(Y_train, y_pred_train)
auc_test1 = roc_auc_score(Y_test1, y_pred_test1)
auc_test2 = roc_auc_score(Y_test2, y_pred_test2)

7413/7413 [==============================] - 6s 829us/step
